In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ptbdb_abnormal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_abnormal.csv', header=None)
ptbdb_normal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_normal.csv', header=None)

In [ ]:
frames = [ptbdb_abnormal_data, ptbdb_normal_data]

ptb_data = pd.concat(frames,keys=["abnormal","normal"])

In [ ]:
ptb_data.loc['normal']

In [ ]:
ptb_data.loc['abnormal']

In [ ]:
msk = np.random.rand(len(ptb_data)) < 0.8

train_ptb = ptb_data[msk]

test_ptb = ptb_data[~msk]

In [ ]:
from keras.utils import to_categorical
ptb_train_X = train_ptb.loc[:, train_ptb.columns != 187]
ptb_train_y = train_ptb.loc[:, train_ptb.columns == 187]
ptb_train_y = to_categorical(ptb_train_y)
ptb_testX = test_ptb.loc[:, test_ptb.columns != 187]
ptb_testy = test_ptb.loc[:, test_ptb.columns == 187]
ptb_testy = to_categorical(ptb_testy)

In [ ]:
print("ptb_train_X shape=" +str(ptb_train_X.shape))
print("ptb_train_y shape=" +str(ptb_train_y.shape))
print("ptb_testX shape=" +str(ptb_testX.shape))
print("ptb_testy shape=" +str(ptb_testy.shape))

In [ ]:
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Embedding, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, AveragePooling2D, MaxPooling2D, MaxPool1D, ZeroPadding1D, GlobalMaxPooling2D, GlobalAveragePooling2D, LSTM, SpatialDropout1D
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.utils import plot_model
from keras.applications.inception_v3 import InceptionV3
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.layers.merge import concatenate

# Simple ANN

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(50, activation='relu', input_shape=(187,)))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(Dense(50, activation='relu'))
ann_model.add(Dense(2, activation='softmax'))

In [ ]:
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ann_model.summary()

In [ ]:
plot_model(ann_model)

In [ ]:
ann_model_history = ann_model.fit(ptb_train_X, ptb_train_y, epochs=100, batch_size = 100, validation_data = (ptb_testX, ptb_testy))

In [ ]:
plt.plot(ann_model_history.history['accuracy'])
plt.plot(ann_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(ann_model_history.history['loss'])
plt.plot(ann_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in ptb_testy:
    y_true.append(np.argmax(element))
prediction_proba=ann_model.predict(ptb_testX)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
ann_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(ann_model_cf_matrix/np.sum(ann_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')

# CNN

# LENET-5

In [ ]:
from sklearn.utils import shuffle
train_ptb[187] = train_ptb[187].astype('float')
test_ptb[187] = test_ptb[187].astype('float')
X_train = np.array(train_ptb.iloc[:, :187])
X_test = np.array(test_ptb.iloc[:, :187])
y_train = np.array(train_ptb[187])
y_test = np.array(test_ptb[187])
X_train, y_train = shuffle(X_train, y_train, random_state = 101)
X_test, y_test = shuffle(X_test, y_test, random_state = 101)
X_train = np.expand_dims(X_train, 2)
X_test = np.expand_dims(X_test, 2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
lenet_5_model=Sequential()

lenet_5_model.add(Conv1D(filters=6, kernel_size=3, padding='same', activation='relu', input_shape=(187,1)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

lenet_5_model.add(Conv1D(filters=16, strides=1, kernel_size=5, activation='relu'))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

lenet_5_model.add(Flatten())

lenet_5_model.add(Dense(64, activation='relu'))

lenet_5_model.add(Dense(32, activation='relu'))

lenet_5_model.add(Dense(2, activation = 'sigmoid'))


In [ ]:
lenet_5_model.summary()

In [ ]:
plot_model(lenet_5_model)

In [ ]:
lenet_5_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lenet_5_model_history = lenet_5_model.fit(X_train, y_train, epochs = 100, batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(lenet_5_model_history.history['accuracy'])
plt.plot(lenet_5_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(lenet_5_model_history.history['loss'])
plt.plot(lenet_5_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=lenet_5_model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
lenet_5_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(lenet_5_model_cf_matrix/np.sum(lenet_5_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')

# AlexNet

In [ ]:
alexNet_model=Sequential()

alexNet_model.add(Conv1D(filters=96, activation='relu', kernel_size=11, strides=4, input_shape=(187,1)))
alexNet_model.add(BatchNormalization())
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Conv1D(filters=384, padding='same', kernel_size=3, activation='relu'))
alexNet_model.add(Conv1D(filters=384, kernel_size=3, activation='relu'))
alexNet_model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
alexNet_model.add(BatchNormalization())
alexNet_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

alexNet_model.add(Flatten())
alexNet_model.add(Dense(4096, activation='relu'))
alexNet_model.add(Dropout(0.4))
alexNet_model.add(Dense(4096, activation='relu'))
alexNet_model.add(Dropout(0.4))
alexNet_model.add(Dense(2, activation='softmax'))

In [ ]:
alexNet_model.summary()

In [ ]:
plot_model(alexNet_model)

In [ ]:
alexNet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
alexNet_model_history = alexNet_model.fit(X_train, y_train, epochs = 100, batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(alexNet_model_history.history['accuracy'])
plt.plot(alexNet_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(alexNet_model_history.history['loss'])
plt.plot(alexNet_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=alexNet_model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
alexNet_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(alexNet_model_cf_matrix/np.sum(alexNet_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')

# VGG-16

In [ ]:
vgg_16_model=Sequential()

vgg_16_model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu',  input_shape=(187,1)))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(Conv1D(filters=512, kernel_size=1, activation='relu', padding='same'))
vgg_16_model.add(BatchNormalization())
vgg_16_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))

vgg_16_model.add(Flatten())
vgg_16_model.add(Dense(4096, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(4096, activation='relu'))
vgg_16_model.add(Dropout(0.4))
vgg_16_model.add(Dense(2, activation='softmax'))

In [ ]:
vgg_16_model.summary()

In [ ]:
plot_model(vgg_16_model)

In [ ]:
vgg_16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_16_model_history = vgg_16_model.fit(X_train, y_train, epochs = 100, batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(vgg_16_model_history.history['accuracy'])
plt.plot(vgg_16_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(vgg_16_model_history.history['loss'])
plt.plot(vgg_16_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=vgg_16_model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
vgg_16_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(vgg_16_model_cf_matrix/np.sum(vgg_16_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')

# ResNet50

In [ ]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv1D(filters = F1, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(filters = F2, kernel_size = f, activation='relu', strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(filters = F3, kernel_size = 1, activation='relu', strides = 1, padding = 'valid')(X)
    X = BatchNormalization()(X)

    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def convolutional_block(X, f, filters, s = 2):
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv1D(F1, 1, activation='relu', strides = s)(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(F2, f, activation='relu', strides = 1,padding = 'same')(X)
    X = BatchNormalization()(X)

    X = Conv1D(F3, 1, strides = 1)(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv1D(F3, 1, strides = s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape = (187,1)):
    
    X_input = Input(input_shape)

    X = ZeroPadding1D(3)(X_input)
    
    X = Conv1D(64, 7, activation='relu', strides = 2)(X)
    X = BatchNormalization()(X)
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)
    
    X = Flatten()(X)
    X = Dense(2,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
resNet50_model = ResNet50(input_shape = (187,1))

In [ ]:
resNet50_model.summary()

In [ ]:
plot_model(resNet50_model)

In [ ]:
resNet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
resNet50_model_history = resNet50_model.fit(X_train, y_train, epochs = 100, batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(resNet50_model_history.history['accuracy'])
plt.plot(resNet50_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(resNet50_model_history.history['loss'])
plt.plot(resNet50_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=resNet50_model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
resNet50_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(resNet50_model_cf_matrix/np.sum(resNet50_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')

# Inception

In [ ]:
def inception_block(prev_layer):
    
    conv1=Conv1D(filters = 64, kernel_size = 1, activation='relu', padding = 'same')(prev_layer)
    
    conv3=Conv1D(filters = 64, kernel_size = 1, activation='relu', padding = 'same')(prev_layer)
    conv3=Conv1D(filters = 64, kernel_size = 3, activation='relu', padding = 'same')(conv3)
    
    conv5=Conv1D(filters = 64, kernel_size = 1, activation='relu', padding = 'same')(prev_layer)
    conv5=Conv1D(filters = 64, kernel_size = 5, activation='relu', padding = 'same')(conv5)
    
    pool= MaxPool1D(pool_size=3, strides=1, padding='same')(prev_layer)
    convmax=Conv1D(filters = 64, kernel_size = 1, activation='relu', padding = 'same')(pool)
    
    layer_out = concatenate([conv1, conv3, conv5, convmax], axis=1)
    return layer_out

In [ ]:
def inception_model(input_shape):
    X_input=Input(input_shape)
    
    X = Conv1D(filters = 64, kernel_size = 7, activation='relu', padding = 'same')(X_input)
    X = MaxPool1D(pool_size=3, strides=2, padding='same')(X)
    
    X = Conv1D(filters = 64, kernel_size = 1, activation='relu', padding = 'same')(X)
    
    X = inception_block(X)
    X = inception_block(X)
    X = inception_block(X)
    X = inception_block(X)
    
    X = MaxPool1D(pool_size=7, strides=2, padding='same')(X)
    
    X = Flatten()(X)
    X = Dense(2,activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X, name='Inception')
    
    return model

In [ ]:
inception_model = inception_model(input_shape = (187,1))

In [ ]:
inception_model.summary()

In [ ]:
plot_model(inception_model)

In [ ]:
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
inception_model_history = inception_model.fit(X_train, y_train, epochs = 100, batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(inception_model_history.history['accuracy'])
plt.plot(inception_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(inception_model_history.history['loss'])
plt.plot(inception_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=inception_model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
inception_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(inception_model_cf_matrix/np.sum(inception_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')

# RNN

# LSTM

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(187,1)))
lstm_model.add(Dense(128, activation = 'relu'))
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(2, activation = 'softmax'))

In [ ]:
lstm_model.summary()

In [ ]:
plot_model(lstm_model)

In [ ]:
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lstm_model_history = lstm_model.fit(X_train, y_train, epochs = 100, batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(lstm_model_history.history['accuracy'])
plt.plot(lstm_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(lstm_model_history.history['loss'])
plt.plot(lstm_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=lstm_model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)

In [ ]:
lstm_model_cf_matrix = confusion_matrix(y_true, prediction)
sns.heatmap(lstm_model_cf_matrix/np.sum(lstm_model_cf_matrix), annot=True,fmt='.3%', cmap='Blues')